# circlesetter.py - STAHR Setting Circles Helper Script

Based on [circlesetter.py](https://drive.google.com/file/d/1c9LbPeNd9SPHVqhoNY9Ls6KNYC2206Az/view) by Casey Murray '25 for use with the setting circles of the Michael Telescope. Contact stahrgazers@gmail.com with questions, improvements, or bug reports. Feel free to modify and share, but please credit STAHR, the Student Astronomers at Harvard-Radcliffe.

v1.3, Sep 8 2024

In [6]:
import requests
import argparse
import numpy as np
from datetime import datetime, timezone

## Configuration

Instead of command-line arguments, you can set the configuration variables in the next cell.

In [7]:
# Verbose output
verbose = True

# Write to file path (e.g., 'output.txt') or leave as empty string
write = ''

# Calculate distances from the Moon
mooncalc = False

# Lists for navigation objects and targets
targets = []
navobjs = []

# Time for observation. Use 'now' for current time or 'YYYY-MM-DD HH:MM'
time_input = 'now'

# Navigation objects and targets
navobjs_input = ['Arcturus']
targets_input = ['M51']

# Update functionality (advanced)
update_input = []

## Function Definitions

In [8]:
# Checks that a time is validly formatted. Returns true if yes, false otherwise
def timecheck(test):
    horizons_url = "https://ssd.jpl.nasa.gov/api/horizons.api?format=text&COMMAND='301'&MAKE_EPHEM='YES'&EPHEM_TYPE='OBSERVER'&CENTER='500@399'&TLIST='"+ test +"'&QUANTITIES='1'&REF_SYSTEM='B1950'"
    horizons = requests.get(horizons_url).text
    lines = horizons.split("\n")
    for line in lines:
        if "$$SOE" in line:
            return True
    return False

# Main object resolution helper method, using the HEASARC Coordinate Converter (SIMBAD) and JPL Horizons. The meat and potatoes of this script.
# object: name of the input object (string)
# navigating: add to navigation objects
# targeting: add to targets
# verbose: print outputs
# usedef: do not ask for time
# bypass: go straight to Horizons
def resolve(object, navigating, targeting, verbose, usedef, bypass=False):
    resolved = False
    global time

    # Special processing for objects input directly with a minor planet designation
    if object.endswith(';'):
        object = object[:-1] + '%3B'
        if not usedef:
            default = time
            string = "Input UT observation time [" + default + "]: "
            time = input(string)
            if time == '':
                time = default
        if verbose and usedef:
            alert = "Searching for ephemeris at " + time + " UT."
            print(alert)

        horizons_url = "https://ssd.jpl.nasa.gov/api/horizons.api?format=text&COMMAND='" + object + "'&MAKE_EPHEM='YES'&EPHEM_TYPE='OBSERVER'&CENTER='500@399'&TLIST='"+ time +"'&QUANTITIES='1'&REF_SYSTEM='B1950'"
        horizons = requests.get(horizons_url).text
        lines = horizons.split("\n")
        ephemeris = ''

        for i in range(len(lines)):
            if i == 4:
                object = lines[i].split()[2]

            if lines[i].find("$$SOE") != -1:
                ephemeris = lines[i+1][30:53].split()
                ra = ephemeris[0] + ':' + ephemeris[1] + ':' + ephemeris[2]
                dec = ephemeris[3] + ':' + ephemeris[4] + ':' + ephemeris[5]
                if verbose: 
                    print("Object resolved in Horizons:",object)
                resolved = True
                break
    
    # Check that not redundant
    found = False
    if len(navobjs) > 0:
        for i in range(len(navobjs)):
            if object.lower() == navobjs[i][0].lower():
                found = True
                print("Navigation object already exists:",object)
                return
    if not found and len(targets) > 0:
        for i in range(len(targets)):
            if object.lower() == targets[i][0].lower():
                print("Target already exists:",object)
                return

    # First check: HEASARC Coordinate Converter, which has SIMBAD built in
    if not bypass:
        heasarc_url = 'https://heasarc.gsfc.nasa.gov/cgi-bin/Tools/convcoord/convcoord.pl?CoordVal=' + object + '&CoordType=B1950&Resolver=GRB%2FSIMBAD%2BSesame%2FNED&NoCache=on&Epoch='
        heasarc = requests.get(heasarc_url).text
        lines = heasarc.split("\n")

        for i in range(len(lines)):
            if lines[i] == '    <th> Lat</th>':
                if verbose: 
                    print('Object resolved in SIMBAD:',object.title())
                ra = lines[i+4].split('<td>')[1].split('&nbsp;')
                dec = lines[i+5].split('<td>')[1].split('&nbsp;')

                ra = ra[0].strip() + ':' + ra[1] + ':' + ra[2].split('</td>')[0].strip()
                dec = dec[0].strip() + ':' + dec[1] + ':' + dec[2].split('</td>')[0].strip()
                
                if dec.find('-') == -1:
                    dec = '+' + dec
                
                resolved = True
                break
            elif lines[i].find('Unable to resolve object/parse coordinates') != -1:
                break
                # It's probably a planet, check the JPL Horizons Ephemeris Generator

    # Second check: Horizons, which may have multiple search results
    if not resolved:
        if not usedef and not bypass:
            default = time
            string = "Object not resolved in SIMBAD: " + object + ". Input UT observation time [" + default + "]: "
            time = input(string)
            if time == '':
                time = default
            elif 'now' in time.lower().strip():
                time = str(datetime.now(timezone.utc)).split()
                time = time[0] + ' ' + time[1].split('+')[0]
        elif not usedef:
            default = time
            string = "Input UT observation time [" + default + "]: "
            time = input(string)
            if time == '':
                time = default
            elif 'now' in time.lower().strip():
                time = str(datetime.now(timezone.utc)).split()
                time = time[0] + ' ' + time[1].split('+')[0]
        if verbose and usedef and not bypass:
            string = "Object not resolved in SIMBAD: " + object + ". Searching for ephemeris at " + time + " UT."
            print(string)
        elif verbose and usedef:
            string = "Searching for lunar ephemeris at " + time + " UT."
            print(string)

        horizons_url = "https://ssd.jpl.nasa.gov/api/horizons.api?format=text&COMMAND='" + object + "'&MAKE_EPHEM='YES'&EPHEM_TYPE='OBSERVER'&CENTER='500@399'&TLIST='"+ time +"'&QUANTITIES='1'&REF_SYSTEM='B1950'"
        horizons = requests.get(horizons_url).text
        lines = horizons.split("\n")
        ephemeris = ''

        multiple = False
        perturbed = False
        orbits = []
        for i in range(len(lines)):
            if 'Multiple' in lines[i]:
                multiple = True
            elif 'Matching small-bodies' in lines[i]:
                perturbed = True

            if multiple:
                if lines[i][11:45].strip().lower() == object.lower():
                    objid = lines[i][:10].strip()
                    horizons_url = "https://ssd.jpl.nasa.gov/api/horizons.api?format=text&COMMAND='" + objid + "'&MAKE_EPHEM='YES'&EPHEM_TYPE='OBSERVER'&CENTER='500@399'&TLIST='"+ time +"'&QUANTITIES='1'&REF_SYSTEM='B1950'"
                    horizons = requests.get(horizons_url).text
                    lines = horizons.split("\n")
                    break
            elif perturbed:
                if len(lines[i].split()) >= 3 and lines[i].split()[2].lower() == object.lower():
                    orbits.append(lines[i].split()[0])
                elif 'matches' in lines[i]:
                    break

            if lines[i].find("$$SOE") != -1:
                ephemeris = lines[i+1][30:53].split()
                ra = ephemeris[0] + ':' + ephemeris[1] + ':' + ephemeris[2]
                dec = ephemeris[3] + ':' + ephemeris[4] + ':' + ephemeris[5]
                if verbose:
                    print("Object resolved in Horizons:",object.title())
                resolved = True
                break

        if perturbed and len(orbits) > 0:
            horizons_url = "https://ssd.jpl.nasa.gov/api/horizons.api?format=text&COMMAND='" + orbits[len(orbits)-1] + '%3B' + "'&MAKE_EPHEM='YES'&EPHEM_TYPE='OBSERVER'&CENTER='500@399'&TLIST='"+ time +"'&QUANTITIES='1'&REF_SYSTEM='B1950'"
            horizons = requests.get(horizons_url).text
            lines = horizons.split("\n")

    # Third check: Get the actual search result we want from Horions
    if not resolved and (multiple or perturbed):
        for i in range(len(lines)):
            if lines[i].find("$$SOE") != -1:
                ephemeris = lines[i+1][30:53].split()
                ra = ephemeris[0] + ':' + ephemeris[1] + ':' + ephemeris[2]
                dec = ephemeris[3] + ':' + ephemeris[4] + ':' + ephemeris[5]
                if verbose:
                    print("Object resolved in Horizons:",object.title())
                resolved = True
                break

    # Results!
    if not resolved:
        if timecheck(time):
            alert = "Object not resolved: " + object.title() + ". Please try again."
            print(alert)
        else:
            alert = "Provided time is not in YYYY-MM-DD HH:MM format: " + time + ". Please try again."
            print(alert)
            time = str(datetime.now(timezone.utc)).split()[0] + ' 23:59'
    elif targeting:
        targets.append([object.title(),ra,dec,-1])
        if verbose:
            print('  RA:',ra,'\n  DEC:',dec)
    elif navigating:
        navobjs.append([object.title(),ra,dec,-1])
        if verbose:
            print('  RA:',ra,'\n  DEC:',dec)
    else:
        return (ra,dec)

# Output a target's apparent coordinates in 0h0m+0d0m
def update(navcoords,delcoords):
    # Parse the provided coordinates
    try: # 0h0m+0d0m (user-provided)
        if '+' in navcoords:
            navra,navdec = navcoords.split('+')
        elif '-' in navcoords:
            navra,navdec = navcoords.split('-')
            navdec = '-' + navdec
        navra = int(navra[:navra.find('h')])+float(navra[navra.find('h')+1:navra.find('m')])/60
        if navdec.find('-') != -1:
            navdec = -1*((-1*int(navdec[:navdec.find('d')]))+float(navdec[navdec.find('d')+1:navdec.find("m")])/60)
        else:
            navdec = int(navdec[:navdec.find('d')])+float(navdec[navdec.find('d')+1:navdec.find("m")])/60
    except Exception: # 0h0m +0°0' (from a file)
        try:
            navra,navdec = navcoords.split()
            navra = int(navra[:navra.find('h')])+float(navra[navra.find('h')+1:navra.find('m')])/60
            if navdec.find('-') != -1:
                navdec = -1*((-1*int(navdec[:navdec.find('°')]))+float(navdec[navdec.find('°')+1:navdec.find("'")])/60)
            else:
                navdec = int(navdec[:navdec.find('°')])+float(navdec[navdec.find('°')+1:navdec.find("'")])/60
        except Exception: # If the user screws up
            string = "Error parsing input coordinates. Format: 0h0m+0d0m. Provided coordinates: " + navcoords
            print(string)
            return False

    # Confirm that the coordinates are validly formatted
    if delcoords == 0:
        return True

    delra,deldec = delcoords.split()
    if delra.find('-') != -1:
        delra = -1*((-1*int(delra[:delra.find('h')]))+float(delra[delra.find('h')+1:delra.find('m')])/60)
    else:
        delra = int(delra[:delra.find('h')])+float(delra[delra.find('h')+1:delra.find('m')])/60
    if deldec.find('-') != -1:
        deldec = -1*((-1*int(deldec[:deldec.find('°')]))+float(deldec[deldec.find('°')+1:deldec.find("'")])/60)
    else:
        deldec = int(deldec[:deldec.find('°')])+float(deldec[deldec.find('°')+1:deldec.find("'")])/60

    # Do the math
    ra, dec = navra + delra, navdec + deldec
    if ra > 24:
        ra -= 24
    elif ra < 0:
        ra += 24

    hrs = int(ra)
    mins = np.round((ra - hrs) * 60,1)
    rastr = str(hrs) + 'h' + str(mins) + 'm'

    # Re-parse the declination
    if dec < 0:
        decstr = '-'
        dec = dec * -1
    else:
        decstr = '+'
    
    degs = int(dec)
    mins = int(np.round((dec - degs) * 60,0))
    decstr = decstr + str(degs) + '°' + str(mins) + "'"

    return (rastr + ' ' + decstr)

# Update a file 
def updatefile(file, updateargs, verbose):
    navobjs = []
    lines = []
    output = []

    # Read the header and try to get a time out of it
    line = file.readline()
    try:
        if not usedef:
            global time
            time = line[line.find('for ')+4:line.find(" UT")]
            if verbose:
                print("Found observing time in file:",time)
        elif verbose:
            faketime = line[line.find('for ')+4:line.find(" UT")]
            string = "Found observing time in file: " + faketime + " UT. Overriden in favor of provided time parameter: " + time + " UT."
            print(string)
    except:
        if verbose:
            print("Warning: corrupted file header.")

    # Parse the file
    navigating = False
    targeting = False
    while line != '':
        if 'Navigation Objects:' in line:
            navigating = True
            targeting = False
            line = file.readline()
        elif 'Targets:' in line:
            targeting = True
            navigating = False
            line = file.readline()
        elif 'Target Apparent Positions' in line:
            break
        if navigating:
            navobjs.append(line[line.find('  ')+2:line.find('  ',2)])
        elif targeting:
            lines.append(line[line.find('  ')+2:line.find('  ',2)] + '  ' + line[line.find('('):])
        line = file.readline()

    if len(navobjs) == 0:
        print("No navigation objects in file.")
        return 0,0,0

    if len(lines) == 0:
        print("No targets in file.")
        return 0,0,0

    if updateargs == False:
        for navobj in navobjs:
            resolve(navobj,True,False,verbose,True)
        for line in lines:
            name = line.split()[0]
            resolve(name,False,True,verbose,True)
        return 0,0,0
    elif len(updateargs) == 1:
        navobj = navobjs[0]
        navcoords = updateargs[0]
    else:
        navobj = updateargs[0]
        navcoords = updateargs[1]

    if not update(navcoords,0):
        return 0,0,0

    offset = len(navobj)

    if navobj.lower() not in lines[0].lower():
        print("Specified navigation object not found in file:",navobj)
        return 0,0,0
    else:
        for line in lines:
            index = line.lower().find(navobj.lower())

            name = line.split()[0]
            delcoords = line[index + offset:line.find("'",index+offset)+1]
            if len(delcoords.split()) != 2:
                print("Specified navigation object not found in file:",navobj)
                return 0,0,0
            string = "  " + name + '  ' + update(navcoords,delcoords)
            output.append(string)

            if verbose:
                print("  Updated target:",string)
    
    return navobj,navcoords,output

# Get the right ascension and declination differences between a navigation object and a target (name/ra/dec tuples)
def diffangle(navobj,target):
    navra = navobj[1].split(':')
    navdec = navobj[2].split(':')
    tarra = target[1].split(':')
    tardec = target[2].split(':')

    navra = float(navra[0]) + ((int(navra[1]) + float(navra[2]) / 60.0) / 60.0)
    tarra = float(tarra[0]) + ((int(tarra[1]) + float(tarra[2]) / 60.0) / 60.0)

    # Declination numbers can be (and in about half of cases, are) negative. This can cause parsing issues
    if '-' in navdec[0]:    
        navdec = -1 * ((-1* float(navdec[0])) + ((int(navdec[1]) + float(navdec[2]) / 60.0) / 60.0))
    else:
        navdec = float(navdec[0]) + ((int(navdec[1]) + float(navdec[2]) / 60.0) / 60.0)

    if tardec[0].find('-') != -1:
        tardec = -1 * ((-1* float(tardec[0])) + ((int(tardec[1]) + float(tardec[2]) / 60.0) / 60.0))
    else:
        tardec = float(tardec[0]) + ((float(tardec[1]) + float(tardec[2]) / 60.0) / 60.0)

    deldec = tardec - navdec
    delra = tarra - navra

    # Get the smaller RA slew
    if delra > 12.0:
        delra -= 24.0
    elif delra < -12.0:
        delra += 24.0

    # Re-parse the right ascension
    if delra < 0:
        rastr = '-'
        delra = delra * -1
    else:
        rastr = '+'

    hrs = int(delra)
    mins = np.round((delra - hrs) * 60,1)
    rastr = rastr + str(hrs) + 'h' + str(mins) + 'm'

    # Re-parse the declination
    if deldec < 0:
        decstr = '-'
        deldec = deldec * -1
    else:
        decstr = '+'
    
    degs = int(deldec)
    mins = int(np.round((deldec - degs) * 60,0))
    decstr = decstr + str(degs) + '°' + str(mins) + "'"

    return (rastr + ' ' + decstr)

# Summarize results
def summarize():
    summary = "Navigation Objects:\n"
    if len(navobjs) == 0:
        summary = summary + '  None.\n' 
    for navobj in navobjs:
        summary += '  ' + navobj[0] + '  ' + navobj[1] + ' ' + navobj[2] + '\n'
    
    summary += "Targets:\n"
    if len(targets) == 0:
        summary = summary + '  None.\n' 
    for target in targets:
        found = False
        summary += '  ' + target[0] + '  ' + target[1] + ' ' + target[2]
        if updateobj[0] == 1 and len(navobjs) > 0:
            if not updateobj[1]:
                summary += ' (' + update(updateobj[2],diffangle(navobjs[0],target)) + ')'
                found = True
            else:
                for i in range(len(navobjs)): 
                    if updateobj[1].lower() in navobjs[i][0].lower():
                        summary += ' (' + update(updateobj[2],diffangle(navobjs[i],target)) + ')'
                        found = True
                        break
        if len(navobjs) > 0 and (not updateobj[0] or not found):
            summary += ' ('
            for i in range(len(navobjs)):
                summary += navobjs[i][0] + ' ' + diffangle(navobjs[i],target)
                if i < len(navobjs) -1:
                    summary += ', '
                else:
                    summary += ')'
        if target[3] != -1:
            summary += ' (Moon distance: ' + target[3] + ')'
        summary += '\n'
    
    return summary

# Helper function to convert sexagesimal RA/Dec (split list format) to decimal degrees.
def get_decimal_coords(ra_split, dec_split):
    # Ensure inputs are treated as floats
    ra_h = float(ra_split[0])
    ra_m = float(ra_split[1])
    ra_s = float(ra_split[2])

    dec_d_str = dec_split[0]
    dec_m = float(dec_split[1])
    dec_s = float(dec_split[2])

    # Convert RA to decimal hours, then to degrees
    ra_hours = ra_h + (ra_m / 60.0) + (ra_s / 3600.0)
    ra_deg = ra_hours * 15.0

    # Convert Dec to decimal degrees, handling the sign correctly
    # Handle optional '+' sign and convert to float
    dec_d = float(dec_d_str.replace('+', ''))

    # The sign of the degrees determines the sign of the whole value
    if dec_d_str.startswith('-') or (dec_d < 0):
        # For negative declination, the minutes and seconds increase the magnitude
        dec_deg = dec_d - (dec_m / 60.0) - (dec_s / 3600.0)
    else:
        dec_deg = dec_d + (dec_m / 60.0) + (dec_s / 3600.0)

    return ra_deg, dec_deg

# Calculate moon distance and print (Corrected Version)
def moondist(verbose):
    # Note: resolve relies on global variables (e.g., usedef, time)
    moon = resolve('Moon',False,False,verbose,usedef,True)

    if moon is None:
        if verbose:
            print("Could not resolve Moon for lunar distance calculation.")
        return

    moonra = moon[0].split(':')
    moondec = moon[1].split(':')

    # Get Moon coordinates in decimal degrees and convert to radians
    ra1_deg, dec1_deg = get_decimal_coords(moonra, moondec)
    ra1_rad = np.radians(ra1_deg)
    dec1_rad = np.radians(dec1_deg)

    if verbose:
        print("Calculating lunar separations:")

    for target in targets:
        tarra = target[1].split(':')
        tardec = target[2].split(':')

        # Get Target coordinates in decimal degrees and convert to radians
        ra2_deg, dec2_deg = get_decimal_coords(tarra, tardec)
        ra2_rad = np.radians(ra2_deg)
        dec2_rad = np.radians(dec2_deg)

        # Spherical Law of Cosines for angular separation
        # cos(a) = sin(dec1)sin(dec2) + cos(dec1)cos(dec2)cos(ra1-ra2)
        cos_a = (np.sin(dec1_rad) * np.sin(dec2_rad) +
                 np.cos(dec1_rad) * np.cos(dec2_rad) * np.cos(ra1_rad - ra2_rad))

        # Handle potential floating point errors by clipping the value to [-1.0, 1.0]
        cos_a = np.clip(cos_a, -1.0, 1.0)

        # Calculate the angle
        angular_rad = np.arccos(cos_a)
        angular_deg = np.round(np.degrees(angular_rad), 1)

        target[3] = str(angular_deg) + "°"

        if verbose:
            string = "  " + target[0] + ": " + str(angular_deg) + "°"
            print(string)

## Main Execution

In [9]:
# Initialize time
if 'now' in time_input.lower().strip():
    time = str(datetime.now(timezone.utc)).split()
    time = time[0] + ' ' + time[1].split('+')[0]
    usedef = True
elif time_input != '':
    time = time_input
    if timecheck(time):
        usedef = True
    else:
        print("Improper time format. Use either 'now' for the current time or specify a UTC time in YYYY-MM-DD HH:MM.")
        time = str(datetime.now(timezone.utc)).split()[0] + ' 23:59'
        usedef = False
else:
    time = str(datetime.now(timezone.utc)).split()[0] + ' 23:59'
    usedef = False

# Load navigation objects and targets into our lists
for navobj in navobjs_input:
    resolve(navobj,True,False,verbose,usedef,False)
for target in targets_input:
    resolve(target,False,True,verbose,usedef,False)

# Run the apparent coordinates calculation routine
updateobj = (0,1)
if len(update_input) > 0:
    if '.txt' in update_input[0] and len(update_input) >= 2:
        try:
            with open(update_input[0],'r') as file:
                if verbose:
                    print("Loading file:",update_input[0])
                navobj, navcoords, updates = updatefile(file,update_input[1:],verbose)
                if updates != 0:
                    with open(update_input[0],'a') as file:
                        string = "Target Apparent Positions (" + navobj + " " + navcoords + "):\n"
                        file.write(string)
                        for update in updates:
                            file.write(update + "\n")
                if verbose and navobj != 0:
                    string = "Saved results to " + update_input[0] + "."
                    print(string)
        except FileNotFoundError:
            print("Could not find file:",update_input[0])
    elif len(update_input) == 2:
        coords = update_input[1]
        if update(coords,0):
            updateobj = (1, update_input[0], coords)
        else:
            print("Usage of update_input incorrect. Usage: ['.txt file', 'navigation object', 'coords']")
    elif '.txt' in update_input[0]:
        try:
            with open(update_input[0],'r') as file:
                if verbose:
                    print("Loading file:",update_input[0])
                updatefile(file,False,verbose)
        except FileNotFoundError:
            print("Could not find file:",update_input[0])
    elif len(update_input) == 1:
        coords = update_input[0]
        if update(coords,0):
            updateobj = (1, False, update_input[0])
        else:
            print("Usage of update_input incorrect. Usage: ['.txt file', 'navigation object', 'coords']")

# Calculate lunar separations
if mooncalc:
    if len(targets) > 0:
        moondist(verbose)
    else:
        print("No target objects to calculate.")

# Print summary
if verbose:
    print('')
    print(summarize())

# Write to file
if write != '':
    try:
        with open(write,'w') as file:
            file.write('Observing report for ' + time + " UT\n")
            file.write(summarize())
            if verbose:
                text = "Saved output to " + write + "."
                print(text)
    except Exception:
        print("Error saving file:",write)

Object resolved in SIMBAD: Arcturus
  RA: 14:13:18.86 
  DEC: +19:24:51.1
Object resolved in SIMBAD: M51
  RA: 13:27:46.32 
  DEC: +47:27:10.6

Navigation Objects:
  Arcturus  14:13:18.86 +19:24:51.1
Targets:
  M51  13:27:46.32 +47:27:10.6 (Arcturus -0h45.5m +28°2')

